In [1]:
import sys
import os
import pandas as pd
import numpy as np
from keras.models import model_from_json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# Load the emotion detection model
model_emotion = model_from_json(open("fer.json", "r").read())
model_emotion.load_weights('fer.h5')

In [3]:
# Load the personality dataset
df_personality = pd.read_csv('emotion_personality_dataset.csv')

In [4]:
# Map emotions to Big Five personality traits
emotion_to_personality = {
    'Happy': {'Extraversion': 1, 'Neuroticism': 0, 'Agreeableness': 0, 'Openness': 0, 'Conscientiousness': 0},
    'Sad': {'Extraversion': 0, 'Neuroticism': 1, 'Agreeableness': 0, 'Openness': 0, 'Conscientiousness': 0},
    'Angry': {'Extraversion': 0, 'Neuroticism': 0, 'Agreeableness': -1, 'Openness': 0, 'Conscientiousness': 0},
    'Neutral': {'Extraversion': 0, 'Neuroticism': 0, 'Agreeableness': 0, 'Openness': 1, 'Conscientiousness': 0},
    'Fear': {'Extraversion': 0, 'Neuroticism': 1, 'Agreeableness': 0, 'Openness': 0, 'Conscientiousness': 0},
    'Surprise': {'Extraversion': 0, 'Neuroticism': 0, 'Agreeableness': 0, 'Openness': 1, 'Conscientiousness': 0},
    'Disgust': {'Extraversion': 0, 'Neuroticism': 0, 'Agreeableness': -1, 'Openness': 0, 'Conscientiousness': 0}
}

In [5]:
# Function to predict personality traits based on detected emotions
def predict_personality(img_pixels):
    # Predict emotion using the emotion detection model
    predictions_emotion = model_emotion.predict(img_pixels)
    max_index = np.argmax(predictions_emotion[0])
    emotions = ['Happy', 'Sad', 'Angry', 'Neutral', 'Fear', 'Surprise', 'Disgust']
    detected_emotion = emotions[max_index]

    # Map detected emotion to personality trait
    personality_trait = emotion_to_personality.get(detected_emotion, 'Unknown')

    return personality_trait

In [6]:
import cv2
import time
from keras.preprocessing import image
#Size for face detection
size = 4
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Capture video from the default webcam
cap = cv2.VideoCapture(0)
now = time.time()
future = now + 10  # Set the duration of the video capture (10 seconds in this case)

while True:
    ret, im = cap.read()
    if not ret:
        continue

    gray_img = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    faces_detected = classifier.detectMultiScale(gray_img, 1.32, 5)

    for (x, y, w, h) in faces_detected:
        cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), thickness=7)
        roi_gray = gray_img[y:y + w, x:x + h]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis=0)
        img_pixels /= 255
        # Predict personality trait based on detected emotion
        predicted_trait = predict_personality(img_pixels)

        font = cv2.FONT_HERSHEY_TRIPLEX
        cv2.putText(im, f'Predicted Trait: {predicted_trait}', (x + h, y - 30), font, 1, (255, 255, 255), 2)

    cv2.imshow('Face detection and Personality prediction', im)
    key = cv2.waitKey(5) & 0xff

    if time.time() > future:
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 33ms/step
